In [2]:
import numpy as np
import pandas as pd
import os
import glob

import netCDF4 as nc
import subprocess as sp

In [3]:
# from config import site_name
SITENAME = 'tuolumne'
name = 'snowdepth'
formato = 'tifs'
years = [2015,2017]

In [4]:
# ------------------------------------------------------------------------------
# Basic Functions
# Function 1 - clip tif to shape

def tif_to_nc(input, output, shp=None):

    # if shp is None: #no shapefile, this just converts tif to netCDF
    #     sp.check_output("gdal_translate -of netCDF {} {}".format(input, output), shell=True)

    if shp != None:    # if there is a shapefile, generate tmp_output, clip it, call it input
        output_tmp = output.split('.')[0]+'_tmp.tif' #parses output name and adds tmp.tif file
        sp.check_output("gdalwarp -of GTiff -dstnodata None -cutline {} -crop_to_cutline {} {}".format(shp, input, output_tmp), shell=True)
    else: #if no shapefile is called, just convert to netCDF
        output_tmp = input

    sp.check_output("gdal_translate -of netCDF {} {}".format(output_tmp, output), shell=True)

    if shp != None:
        sp.check_output('rm {}'.format(output_tmp), shell= True) #removes output_tmp items

## Call Functions

In [12]:
for year in sorted(years):
    path_to_data = f'/srv/shared/deep_stac/data/{name}/{formato}/{year}'
    files = os.listdir(path_to_data)

    for f in files:
        llave = f.split('_')[0]
        new_path = f'/srv/shared/deep_stac/data/{name}/netcdf/{year}'
        out_fname = os.path.join(new_path, f'{llave}_SUPERsnow_depth_3m_int.nc')
        # CALL FUNCTION
        tif_to_nc(os.path.join(path_to_data, f), out_fname, shp=None)